In [25]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding


In [26]:
df = pd.read_csv("processed_reviews.csv")

# Check class distribution
print(df['sentiment'].value_counts())

sentiment
2    9022
1     772
0     467
Name: count, dtype: int64


In [27]:
x = list(df['reviews'])
y = list(df['sentiment'])

In [28]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42,
    stratify=df['sentiment']
)

In [29]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [30]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [31]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-31-1693411593.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.610600
100,0.422800
150,0.443700
200,0.401800
250,0.386000
300,0.392100
350,0.335400
400,0.382400
450,0.359500
500,0.315100


TrainOutput(global_step=1539, training_loss=0.2607624296866275, metrics={'train_runtime': 612.098, 'train_samples_per_second': 40.229, 'train_steps_per_second': 2.514, 'total_flos': 1619726199533568.0, 'train_loss': 0.2607624296866275, 'epoch': 3.0})

In [35]:
predictions=trainer.predict(test_dataset)
preds=torch.argmax(torch.tensor(predictions.predictions),axis=1)

print("\nClassification Report: \n")
print(classification_report(test_labels,preds.numpy(),target_names=['Negative','Neutral','Postitive']))


Classification Report: 

              precision    recall  f1-score   support

    Negative       0.65      0.48      0.56        93
     Neutral       0.38      0.37      0.38       155
   Postitive       0.95      0.96      0.95      1805

    accuracy                           0.89      2053
   macro avg       0.66      0.61      0.63      2053
weighted avg       0.89      0.89      0.89      2053



In [38]:
model.eval()

text="This is a average product."
inputs=tokenizer(text,return_tensors='pt',truncation=True,padding=True,max_length=128)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs={key: val.to(device) for key, val in inputs.items()}
model.to(device)

outputs=model(**inputs)
logits=outputs.logits
predicted_class_id=torch.argmax(logits,dim=1).item()

print(predicted_class_id)

1
